### GPU

In [17]:
!nvidia-smi

Fri Aug 11 22:22:26 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.54       Driver Version: 526.56       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0 Off |                  N/A |
| N/A   48C    P8     4W /  N/A |    858MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

**计算设备**

In [18]:
import torch
from torch import nn

torch.device("cpu"), torch.cuda.device("cuda"), torch.cuda.device("cuda:1")

(device(type='cpu'),
 <torch.cuda.device at 0x7f1b6ff2e850>)

**查询可用GPU的数量**

In [19]:
torch.cuda.device_count()

1

**这两个函数允许我们在请求的GPU不存在的情况下运行代码**

In [20]:
def try_gpu(i=0):
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f"cuda:{i}")
    return torch.device("cpu")


def try_all_gpus():
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f"cuda:{i}") for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

**查询张量所在的设备**

In [21]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

**存储在GPU上**

In [22]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

**第二个GPU上创建一个随机张量**

In [23]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y

tensor([[0.8791, 0.7037, 0.3477],
        [0.8778, 0.9328, 0.6157]])

**要计算`X + Y`，我们需要决定在哪里执行这个操作**

In [25]:
Z = X.cuda(1)   # 将张量X从GPU 0 copy到GPU 1
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


**现在数据都在同一个GPU上（`Z` 和 `Y` 都在），我们可以将它们相加**

如果数据在不同的`device`，来回挪动东西就会很麻烦，故而pytorch直接报错，不允许不同设备的数据做运算

In [32]:
Y + Z

tensor([[1.8791, 1.7037, 1.3477],
        [1.8778, 1.9328, 1.6157]], device='cuda:0')

**检查一个张量 `Z` 是否已经被移动到 GPU 设备上的特定 GPU 设备编号（GPU 0）**

**检查原始张量 Z 是否与经过 .cuda(0) 移动到 GPU 0 后的张量相同。请注意，这里的检查只会考虑张量的身份（内存地址），而不会考虑内容是否相同。**

In [46]:
Z.cuda(0) is Z

True

--------------------------

**神经网络与GPU**

In [48]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

net(X)

tensor([[-0.1724],
        [-0.1724]], device='cuda:0', grad_fn=<AddmmBackward0>)

**确认模型参数存储在同一个GPU上**

In [55]:
net[0].weight.data.device, X.device

(device(type='cuda', index=0), device(type='cuda', index=0))